In [1]:
!ls|grep Rus

RusDictionary.txt


In [2]:
with open('RusDictionary.txt',encoding='utf-8') as f:
    words = f.read().split('\n')

# Python как аллюзия на жизнь

Задача: В одну строку кода вывести все самые длинные слова из списка слов, и при этом сделать так, чтобы асимптотика алгоритма была лучше, чем O(N2). Проблема заключается в том, что код ниже возвращает только один элемент.

In [3]:
print(max(words,key=len))

частнопредпринимательский


Это нас не устраивает, так как вполне вероятно, что есть слово, которое будет иметь ту же длину.

## Однострочное решение(наивное)

Самое простое решение выглядит следующим образом:

In [4]:
%%time
print([word for word in tqdm(words) if len(word) == max(len(word) for word in words)])


NameError: name 'tqdm' is not defined

Это решение работает следующим образом:

Для каждого слова в списке проверяется его длина.
Если длина слова равна максимальной длине, то слово добавляется в список результатов.
Это решение работает за O(N2), потому что оно должно проверить каждое слово в списке. tqdm здесь только для отслеживания степени выполнения задачи.

А теперь посчитаем количество элементов массива

In [5]:
len(words)

93384

10 минут для массива из 100000 крайне плохая история, так как увеличение количество элементов приведет к квадратичному росту итераций. Поэтому отойдем от поиска однострочечного решения.

## Две строки это не так плохо

Первое, что приходит на ум для достижения O(n) - это разделить задачу на подзадачи, а именно найти длину самого длинного слова, а далее найти все слова заданной длины

In [6]:
%%time
max_len = max(map(len,words))
print({max_len:tuple(filter(lambda x: len(x) == max_len,words))})

{25: ('частнопредпринимательский', 'электронно-вычислительный')}
CPU times: total: 46.9 ms
Wall time: 28 ms


То есть задача решаема, хотя бы в 2 строки. Следовательно ничего не мешает немного нагло формально решить данную задачу.

## Методы обмана

### Использование точек с запятой вместо новых строк

In [7]:
%%time
max_len = max(map(len,words));print({max_len:tuple(filter(lambda x: len(x) == max_len,words))})

{25: ('частнопредпринимательский', 'электронно-вычислительный')}
CPU times: total: 31.2 ms
Wall time: 31 ms


Да, мы можем это сделать! И формально это одна строка. Но можно подойти чуть поинтереснее, использовать метод exec

In [8]:
%%time
exec('max_len = max(map(len,words))\nprint({max_len:tuple(filter(lambda x: len(x) == max_len,words))})')

{25: ('частнопредпринимательский', 'электронно-вычислительный')}
CPU times: total: 31.2 ms
Wall time: 45 ms


Метод exec() в Python выполняет динамически созданную программу, которая является либо строкой, либо объектом кода.

## Повышаем маржу моржовыми операторами

In [9]:
%%time
print({(l := max(map(len,words))):tuple(filter(lambda x: len(x) == l,words))})

{25: ('частнопредпринимательский', 'электронно-вычислительный')}
CPU times: total: 31.2 ms
Wall time: 17 ms


Тут также мы объединяем две строки немного нечестным способом. Однако разницы с предыдущими решениями заключается в том, что использование моржовых операторов в Python позволяет присвоить значение переменной и вернуть это значение, поэтому порой можно написать код короче и сделать его более читаемым, и он может быть даже более эффективным с точки зрения вычислений. А потому будем считать это более честным решением данной задачи. В дальнейших решениях я также буду использовать моржовые выражения из-за желания увеличить читаемость кода.

## Функциональное программирование без Reduse? Это как?

Основная проблема функций map, filter является то, что функция применяется поэлементно(можно даже сказать независимо), а потому пройтись с помощью них один раз по массиву не даст нам информации. Можно использовать в качестве функции, которая позволит нам использовать результаты прошлой итерации, например, Reduse.

In [10]:
%%time
from functools import reduce
print(reduce(lambda x,y: {l:[y]} if (l := len(y)) > (key := tuple(x.keys())[0]) else {key:x[key]+[y]} if l == key else x,words,{0:[]}))

{25: ['частнопредпринимательский', 'электронно-вычислительный']}
CPU times: total: 31.2 ms
Wall time: 40 ms


Описать работу данной программы можно с помощью несжатой функции

In [11]:
%%time
def func(x,y):
#     x is {key:list}
#     key - это длина самого длинного слова, которое встретилось на данной итерации
#     list - список элементов данной длины
    ret = None
    if (l:=len(y))>(key := tuple(x.keys())[0]):
        return {l:[y]}
    else:
        if l == key:
            return {key:x[key]+[y]}
        else:
            return x
reduce(func,words,{0:[]})

CPU times: total: 62.5 ms
Wall time: 53 ms


{25: ['частнопредпринимательский', 'электронно-вычислительный']}

Для большей формальности сделаем импорт и вызов в одной строчке кода

In [12]:
%%time
print(__import__("functools").reduce(lambda x,y: {l:[y]} if (l := len(y)) > (key := tuple(x.keys())[0]) else {key:x[key]+[y]} if l == key else x,words,{0:[]}))

{25: ['частнопредпринимательский', 'электронно-вычислительный']}
CPU times: total: 46.9 ms
Wall time: 60 ms


## А почему бы нам не переписать взаимодействие

Если мы не удовлетворены работой стандартных типов данных для какой-либо задачи, всегда можно их просто переписать.

In [32]:
from joblib import Parallel, delayed
import numpy as np
class List(list):
    @staticmethod
    def sum_of_sets(sets,union_set):
        for set in sets:
            union_set = union_set.union(set)
        return union_set
    @staticmethod
    def find_the_max(array):
        l = 0
        arr = set()
        for i in array:
            if l == (m:=len(i)):
                arr.add(i)
            if l < m:
                arr = {i}
                l = m
        return (l,arr)
    def max_str(self,n):
        cores = 8
        data = np.array_split(self,n)
        data = Parallel(n_jobs=n)(delayed(self.find_the_max)(i) for i in data)
        l = max(map(lambda x: x[0], data))
        t = self.sum_of_sets(map(lambda x: x[1],filter(lambda x: x[0] == l, data)),set())
        return t

In [33]:
%%time
l = List(words)
print(l.max_str(8))

{'частнопредпринимательский', 'электронно-вычислительный'}
CPU times: total: 78.1 ms
Wall time: 206 ms


#

## Аллюзия на жизнь

Таким образом, мы видим, что в Питоне не существует решения задачи вывода всех самых длинных слов из списка слов за одну строку с асимптотикой лучше, чем O(N2) без небольших трюков.

Этот факт можно рассматривать как аллюзию на жизнь. В жизни часто бывает так, что самое короткое решение не является оптимальным. Для того, чтобы добиться наилучшего результата, нам часто приходится идти на компромисс и выбирать более длинное решение.

В этом случае, если мы хотим решить задачу за одну строку, мы должны пожертвовать асимптотической сложностью или читаемостью.